# Team

在 AgentChat 中，团队定义了代理组如何协作处理任务。一个团队由一个或多个代理组成，通过接收任务和返回任务结果与您的应用程序交互。
它是有状态的，并在多个任务之间维护上下文。团队使用有状态的终止条件来确定何时停止处理当前任务。

下图显示了团队与您的应用程序之间的关系。

![AgentChat Teams](./agentchat-team.svg)

AgentChat 提供了几个预设团队，它们实现了一个或多个[多代理设计模式](../../core-user-guide/design-patterns/index.md)以简化开发。以下是预设团队列表：

- {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`: 所有参与者共享上下文，并以轮询方式轮流响应。
- {py:class}`~autogen_agentchat.teams.SelectorGroupChat`: 所有参与者共享上下文，并使用基于模型的选择器（可自定义覆盖）来选择下一个响应的代理。
- {py:class}`~autogen_agentchat.teams.Swarm`: 所有参与者共享上下文，并使用 {py:class}`~autogen_agentchat.messages.HandoffMessage` 将控制权传递给下一个代理。

在高层次上，团队 API 包含以下方法：

- {py:meth}`~autogen_agentchat.base.TaskRunner.run`: 处理任务，可以是 {py:class}`str`、{py:class}`~autogen_agentchat.messages.TextMessage` 或 {py:class}`~autogen_agentchat.messages.MultiModalMessage`，并返回 {py:class}`~autogen_agentchat.base.TaskResult`。如果团队尚未重置，任务也可以是 `None` 以继续处理前一个任务。
- {py:meth}`~autogen_agentchat.base.TaskRunner.run_stream`: 与 {py:meth}`~autogen_agentchat.base.TaskRunner.run` 相同，但返回消息的异步生成器和最终任务结果。
- {py:meth}`~autogen_agentchat.base.Team.reset`: 如果下一个任务与前一个任务无关，则重置团队状态。否则，团队可以利用前一个任务的上下文来处理下一个任务。

在本节中，我们将使用 {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat` 团队来介绍 AgentChat 团队 API。

## Round-Robin Group Chat

我们将从创建一个只有一个 {py:class}`~autogen_agentchat.agents.AssistantAgent` 代理和 {py:class}`~autogen_agentchat.task.TextMentionTermination` 终止条件的团队开始，该条件在检测到特定词时停止。

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key="sk-", # Optional if you have an OPENAI_API_KEY env variable set.
)


# Define a tool that gets the weather for a city.
async def get_weather(city: str) -> str:
    """Get the weather for a city."""
    return f"The weather in {city} is 72 degrees and Sunny."


# Create an assistant agent.
weather_agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition.
text_termination = TextMentionTermination("TERMINATE")

# Create a single-agent team.
single_agent_team = RoundRobinGroupChat([weather_agent], termination_condition=text_termination)

### 运行团队

我们接下来调用 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` 方法以通过一个task启动团队.

In [4]:
async def run_team() -> None:
    result = await single_agent_team.run(task="What is the weather in New York?")
    print(result)


# Use `asyncio.run(run_team())` when running in a script.
await run_team()

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?'), ToolCallMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=15), content=[FunctionCall(id='call_yEkJmydaVYUpZMS3b6wfmUkF', arguments='{"city":"New York"}', name='get_weather')]), ToolCallResultMessage(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', call_id='call_yEkJmydaVYUpZMS3b6wfmUkF')]), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=96, completion_tokens=14), content='The weather in New York is currently 72 degrees and sunny.'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=126, completion_tokens=4), content='TERMINATE')], stop_reason="Text 'TERMINATE' mentioned")


团队运行相同的代理，直到满足终止条件。
在这种情况下，当在代理的响应中检测到"TERMINATE"一词时，就满足了终止条件。
当团队停止时，它会返回一个 {py:class}`~autogen_agentchat.base.TaskResult` 对象，其中包含团队中代理产生的所有消息。

### 重置团队

您可以通过调用 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset` 方法来重置团队。
它将清除团队的状态，包括其所有代理的状态。

In [13]:
await single_agent_team.reset()  # Reset the team for the next run.

如果下一个任务与前一个任务无关，通常最好重置团队。
但是，如果下一个任务与前一个任务相关，则不需要重置。

请参阅下面的[恢复团队](#resuming-team)。

### 流式传输团队消息

与代理的 {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` 方法类似，
您可以通过调用 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream` 方法来流式传输团队的消息。
它将返回一个生成器，该生成器会在团队中的代理生成消息时产生这些消息，最后一项将是任务结果。

In [5]:
from autogen_agentchat.base import TaskResult


async def run_team_stream() -> None:
    async for message in single_agent_team.run_stream(task="What is the weather in New York?"):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)


# Use `asyncio.run(run_team_stream())` when running in a script.
await run_team_stream()

source='user' models_usage=None content='What is the weather in New York?'
source='assistant' models_usage=RequestUsage(prompt_tokens=148, completion_tokens=15) content=[FunctionCall(id='call_vKkTpRmeFFclKFZI5Of8NiFA', arguments='{"city":"New York"}', name='get_weather')]
source='assistant' models_usage=None content=[FunctionExecutionResult(content='The weather in New York is 72 degrees and Sunny.', call_id='call_vKkTpRmeFFclKFZI5Of8NiFA')]
source='assistant' models_usage=RequestUsage(prompt_tokens=174, completion_tokens=14) content='The weather in New York is currently 72 degrees and sunny.'
source='assistant' models_usage=RequestUsage(prompt_tokens=204, completion_tokens=4) content='TERMINATE'
Stop Reason: Text 'TERMINATE' mentioned


如上例所示，您可以通过检查 {py:attr}`~autogen_agentchat.base.TaskResult.stop_reason` 属性来获取团队停止的原因。

有一个方便的方法 {py:meth}`~autogen_agentchat.task.Console`，它可以将消息以适当的格式打印到控制台。

In [6]:
from autogen_agentchat.task import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

---------- user ----------
What is the weather in Seattle?
---------- assistant ----------
[FunctionCall(id='call_50RPt8ay50PxbilD3RxXm9Ko', arguments='{"city":"Seattle"}', name='get_weather')]
[Prompt tokens: 69, Completion tokens: 14]
---------- assistant ----------
[FunctionExecutionResult(content='The weather in Seattle is 72 degrees and Sunny.', call_id='call_50RPt8ay50PxbilD3RxXm9Ko')]
---------- assistant ----------
The weather in Seattle is currently 72 degrees and sunny.
[Prompt tokens: 93, Completion tokens: 13]
---------- assistant ----------
TERMINATE
[Prompt tokens: 122, Completion tokens: 4]
---------- Summary ----------
Number of messages: 5
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 284
Total completion tokens: 31
Duration: 2.36 seconds


### 反思模式

现在我们将创建一个包含两个代理的团队，它实现了反思模式，这是一种多代理设计模式，使用评论家代理来评估主要代理的响应。

查看使用 [Core API](../../core-user-guide/design-patterns/reflection.ipynb) 的反思模式如何工作。

在这个例子中，我们将对主要代理和评论家代理都使用 {py:class}`~autogen_agentchat.agents.AssistantAgent` 代理类。
我们将同时使用 {py:class}`~autogen_agentchat.task.TextMentionTermination` 和 {py:class}`~autogen_agentchat.task.MaxMessageTermination` 终止条件来停止团队。

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.task import Console, MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key="sk-", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=termination)

让我们给团队一个写诗的任务，看看代理之间如何互动。

In [8]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)  # Stream the messages to the console.

---------- user ----------
Write a short poem about fall season.
---------- primary ----------
Leaves of amber, gold, and rust,  
Gently drift, as branches trust,  
Crisp whispers dance upon the breeze,  
In the symphony of swaying trees.  

The air turns brisk, a hint of chill,  
Pumpkin scents the windowsill,  
Sweaters wrap in warm embrace,  
As nature shifts at autumn's pace.  

Harvest moons and longest nights,  
Lit with warmth from fire's lights,  
Fall's gentle touch, both bold and sweet,  
Marks the earth beneath our feet.  
[Prompt tokens: 27, Completion tokens: 110]
---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a soothing rhythm. The use of sensory details, like "crisp whispers" and "pumpkin scents," effectively evokes the atmosphere of autumn. The structure flows well, and the rhyme scheme enhances the poem's cohesiveness. 

To enhance your poem further, consider adding a line or two that conveys an emotion

### 恢复团队

让我们在保持前一个任务的上下文的同时，用一个新任务再次运行团队。

In [9]:
# Write the poem in Chinese Tang poetry style.
# Use `asyncio.run(Console(reflection_team.run_stream(task="将这首诗用中文唐诗风格写一遍。")))` when running in a script.
await Console(reflection_team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

---------- user ----------
将这首诗用中文唐诗风格写一遍。
---------- primary ----------
秋叶金黄伴枫红，  
随风轻舞信枝桐。  
微凉入袖秋声起，  
漫洒余晖满林中。  

篱边南瓜香自暖，  
软衣温柔度晨昏。  
月下收成思往事，  
悠悠往昔寄心间。  

满目枯荣循岁月，  
万物变幻自有情。  
片片落叶皆故事，  
新陈代谢展新生。  
[Prompt tokens: 613, Completion tokens: 113]
---------- critic ----------
你的诗作成功地将英文版本转换成了中文唐诗风格，展现了秋天的美丽和情感。你运用了优雅的语言和经典的对仗结构，使得整首诗既有传统韵味，又不失原作的意境。

特别喜欢你在视觉和嗅觉上的描述，比如“秋叶金黄伴枫红”和“篱边南瓜香自暖”，这些都生动地描绘了秋天的特征。情感上的表达也处理得当，“月下收成思往事，悠悠往昔寄心间”很好地抓住了秋天带来的那种怀旧与思索。

个别用词和句子上，可以考虑稍作调整，以增强唐诗的古典韵味和简练表达，但不失为一篇成功的转化作品。

整体来说，你的作品很好地保留了原诗的意象与情感，且赋予了中文诗特有的韵律和风貌。佳作！
[Prompt tokens: 744, Completion tokens: 227]
---------- primary ----------
感谢您的详细评价和鼓励！很高兴听到您对这首诗的欣赏和建议。根据您的反馈，我对部分句子进行了调整，以进一步强化唐诗的古典韵味和简练表达。以下是修改后的版本：

秋叶金风映枫林，  
随风轻舞信枝身。  
微凉入袖秋声至，  
斜阳洒落漫山昕。  

篱畔瓜香暖人意，  
长袖轻拥度朝曛。  
月下思怀当年事，  
点滴往昔驻心神。  

枯荣迭代随天命，  
万物更替总有情。  
片片落叶皆故事，  
新旧交替见新生。  

希望这次的版本更符合唐诗的风格和表达。感谢您的建议，让这首诗更加完善！
[Prompt tokens: 965, Completion tokens: 197]
---------- critic ----------
这个经过调整后的版本确实更贴合唐诗的风格，展现了简练而优雅的古典韵味。你在语言上做出的细微改动

继续另一项任务。

In [10]:
# Write the poem in Spanish.
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write the poem in Spanish.")))` when running in a script.
await Console(reflection_team.run_stream(task="Write the poem in Spanish."))

---------- user ----------
Write the poem in Spanish.
---------- primary ----------
Here is the poem translated into Spanish:

Hojas de ámbar, oro, carmesí,  
Danzo al viento que anuncia así,  
Susurros nítidos en brisa leve,  
Mecen los árboles, danza que se atreve.  

El aire se enfría, suave frescor,  
Calabaza perfuma el entorno en su sabor,  
Jerséis envuelven en cálido abrazo,  
Mientras el otoño avanza con su paso.  

Bajo la luna de cosecha, el recuerdo vive,  
Melodías del pasado, el corazón revive,  
Momentos que duran, tiempos queridos,  
Ecos de cambio, ciclos compartidos.  

La callada transformación, dulce y fuerte,  
Marca la tierra bajo el pie andante,  
En cada hoja, una historia gira,  
De finales y comienzos, la vida conspira.  
[Prompt tokens: 1399, Completion tokens: 185]
---------- critic ----------
Your Spanish translation of the poem effectively retains the essence and imagery of the original version. The vivid descriptions, like "hojas de ámbar, oro, carmesí" a

### 恢复前一个任务

我们可以调用 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` 或 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream` 方法，而无需再次设置 `task` 来恢复前一个任务。团队将从停止的地方继续。

In [11]:
# Use the `asyncio.run(Console(reflection_team.run_stream()))` when running in a script.
await Console(reflection_team.run_stream())

---------- primary ----------
Thank you for your thoughtful review and approval. I appreciate your insights regarding the rhythm and structure. Here's a slight revision to enhance the flow while keeping the imagery and emotional tone intact:

Hojas de ámbar, oro en vuelo,  
Bailan al viento con suave anhelo.  
Susurros frescos en brisa leve,  
Mecen los árboles, el canto se atreve.  

El aire enfría con fresco sabor,  
La calabaza perfuma de amor.  
Jerséis cobijan calidez cercana,  
Mientras el otoño su paso desgrana.  

La luna de cosecha nos hace pensar,  
Melodías pasadas vuelven a sonar.  
Momentos que perduran, tiempos vividos,  
Ecos de cambio, ritmos compartidos.

Transformación callada, dulce y entera,  
Marca la tierra donde la vida espera.  
En cada hoja, una historia canta,  
De fines y principios, la vida encanta.  

I hope this version better captures the melodic essence of the poem. Thank you for your feedback—it continues to guide the refinement process.
[Prompt tokens:

### 暂停等待用户输入

通常，团队需要来自应用程序（即用户）的额外输入才能继续处理任务。我们将展示两种可能的方法：

- 设置最大轮次数，使团队在指定轮次数后停止。
- 使用 {py:class}`~autogen_agentchat.task.HandoffTermination` 终止条件。

您也可以使用自定义终止条件，请参阅 [Termination Conditions](./termination.ipynb).

#### 最大轮次

这是暂停团队等待用户输入的最简单方法。例如，您可以将最大轮次数设置为 1，这样团队会在第一个代理响应后立即停止。当您希望用户持续与团队互动时，这很有用，比如在聊天机器人场景中。

只需在 {py:meth}`~autogen_agentchat.teams.RoundRobinGroupChat` 构造函数中设置 `max_turns` 参数。

```python
team = RoundRobinGroupChat([...], max_turns=1)
```

一旦团队停止，轮次计数将被重置。当您恢复团队时，它将再次从 0 开始。

注意，`max_turn` 是特定于团队类的，目前仅由
{py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`、{py:class}`~autogen_agentchat.teams.SelectorGroupChat` 和 {py:class}`~autogen_agentchat.teams.Swarm` 支持。
当与终止条件一起使用时，团队将在满足任一条件时停止。

#### 使用移交来暂停团队

您可以使用 {py:class}`~autogen_agentchat.task.HandoffTermination` 终止条件来在代理发送 {py:class}`~autogen_agentchat.messages.HandoffMessage` 消息时停止团队。

让我们创建一个包含单个带有移交设置的 {py:class}`~autogen_agentchat.agents.AssistantAgent` 代理的团队。

```{note}
与 {py:class}~autogen_agentchat.agents.AssistantAgent 一起使用的模型必须支持工具调用才能使用移交功能。
```

In [12]:
from autogen_agentchat.agents import AssistantAgent, Handoff
from autogen_agentchat.task import HandoffTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key="sk-", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat([lazy_agent], termination_condition=termination)

让我们用一个需要用户额外输入的任务来运行团队，因为代理没有相关的工具来继续处理任务。

In [13]:
from autogen_agentchat.task import Console

# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="What is the weather in New York?")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="What is the weather in New York?"))

---------- user ----------
What is the weather in New York?
---------- lazy_assistant ----------
[FunctionCall(id='call_TUNN7UUSzSIzkPW4OQs4wENx', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 68, Completion tokens: 11]
---------- lazy_assistant ----------
[FunctionExecutionResult(content='Transfer to user.', call_id='call_TUNN7UUSzSIzkPW4OQs4wENx')]
---------- lazy_assistant ----------
Transfer to user.
---------- Summary ----------
Number of messages: 4
Finish reason: Handoff to user from lazy_assistant detected.
Total prompt tokens: 68
Total completion tokens: 11
Duration: 1.26 seconds


您可以看到团队因为检测到移交消息而停止。
让我们通过提供代理需要的信息来继续团队。

In [14]:
# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="It is raining in New York.")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="It is raining in New York."))

---------- user ----------
It is raining in New York.
---------- lazy_assistant ----------
Thank you for the update! If you have any other questions or need further information, feel free to ask.
[Prompt tokens: 108, Completion tokens: 23]
---------- lazy_assistant ----------
TERMINATE
[Prompt tokens: 138, Completion tokens: 4]
---------- Summary ----------
Number of messages: 3
Finish reason: Text 'TERMINATE' mentioned
Total prompt tokens: 246
Total completion tokens: 27
Duration: 1.52 seconds



```{note}
目前移交终止方法不适用于 {py:class}`~autogen_agentchat.teams.Swarm`。
请继续关注更新。
```
```